In [12]:
import numpy as np
import pickle
import sys
import my_bite_detection_utils as bdu
import my_classification_utils as mcu
import os

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
import my_tensorflow_utils as mtu
import importlib
from sklearn import metrics

In [13]:
importlib.reload(mtu)
importlib.reload(bdu)
importlib.reload(mcu)

<module 'my_classification_utils' from 'C:\\ASM\\Dropbox\\Developments\\Jupyter\\Eating\\bite_detection\\my_classification_utils.py'>

In [14]:
root_path ='C:/ASM/DevData/eating' if "C:" in os.getcwd() else "."
print("Root path: ", root_path)

params = {"x_th": -0, "min_bite_interval": 2*16, "window_size": 6*16}
windows, ssml, labels, features = bdu.get_windows_lab(params)
labels[:, 1] = labels[:, 1] + labels[:, 2]
labels = labels[:, :2]
print(windows.shape, ssml.shape, np.sum(labels, axis=0))

Root path:  C:/ASM/DevData/eating
Creating windows and labels ... x_th: 0, min_bite_interval: 32, window_size: 96
(52477, 96, 6) (52477, 4) [47886  4591]


In [15]:
#Training Parameters
net_params={}
net_params['learning_rate'] = 0.001
net_params['num_epochs'] = 50
net_params['batch_size'] = 128
net_params['keep_prob_val'] = 0.5

In [16]:
params["x_th"] = -.4
for var_th in np.arange(0, 2.1, .25):    
    var_th = int(var_th*100)/100    
    if var_th !=1.0: 
        continue
    
    params["var_th"] = var_th 
    cond = (features[:, 0] <=params["x_th"]) & (features[:, -1] >=params["var_th"]) #& (ssml[:, 0]>=7)
    w = windows[cond]
    s = ssml[cond]
    l = labels[cond]
    f = features[cond]
    print(w.shape, s.shape, np.sum(l, axis=0))
    
    for exclude_subject in range(1,2):
        print("******** Exclude Subject {} *********".format(exclude_subject))
        print(params)
        cond = s[:,0]!=exclude_subject
        train_x = w[cond]
        train_y = l[cond]
        
        cond = s[:,0]==exclude_subject
        test_x = w[cond]        
        test_y = l[cond]

        print("Train Test shapes: ", train_x.shape, test_x.shape)
        print("Train Test labels:", np.sum(train_y, axis=0), np.sum(test_y, axis=0))

        path = root_path+'/outputs/bite_detection_lopo/'+bdu.param_string(params)+"/subject_"+str(exclude_subject)     
        test_pred, train_result, test_result = mtu.train_test_model(train_x, train_y, test_x, test_y, folder_path=path, params=net_params)
        path = path+'/results'
        bdu.create_directory(path)
        np.savetxt(path+'/test_y.csv', test_y, fmt='%.4f', delimiter=',')    
        np.savetxt(path+'/test_prediction.csv', test_pred, fmt='%.4f', delimiter=',')
        np.savetxt(path+'/train_result.csv', train_result, fmt='%.4f', delimiter=',')
        np.savetxt(path+'/test_result.csv', test_result, fmt='%.4f', delimiter=',') 


(13639, 96, 6) (13639, 4) [9719 3920]
******** Exclude Subject 1 *********
{'x_th': -0.4, 'min_bite_interval': 32, 'window_size': 96, 'var_th': 1.0}
Train Test shapes:  (12994, 96, 6) (645, 96, 6)
Train Test labels: [9110 3884] [609  36]
Inside all_axes_net: x_shape, y_shape : [None, 96, 6] [None, 2]
<class 'list'>
Inside one_axis_net:  conv_axis_0 , x_shape [None, 96, 1]
  Axis count:  1
  Conv_1, maxpool_1 shape:  [None, 92, 1, 16] [None, 46, 1, 16]
  Conv_2, maxpool_2 shape:  [None, 42, 1, 32] [None, 21, 1, 32]
  Conv_3, maxpool_3 shape:  [None, 17, 1, 64] [None, 8, 1, 64]
Shape  0 [None, 512]
Inside one_axis_net:  conv_axis_1 , x_shape [None, 96, 1]
  Axis count:  1
  Conv_1, maxpool_1 shape:  [None, 92, 1, 16] [None, 46, 1, 16]
  Conv_2, maxpool_2 shape:  [None, 42, 1, 32] [None, 21, 1, 32]
  Conv_3, maxpool_3 shape:  [None, 17, 1, 64] [None, 8, 1, 64]
Shape  1 [None, 512]
Inside one_axis_net:  conv_axis_2 , x_shape [None, 96, 1]
  Axis count:  1
  Conv_1, maxpool_1 shape:  [None,

  Train loss: 0.6550, acc: 0.7131, pr: 0.5214, rc: 0.4625, sp: 0.8196, f1: 0.4902
                                   tn: 7509, tp: 1801, fn: 2093, fp: 1653
  Test  loss: 0.6895, acc: 0.7783, pr: 0.0720, rc: 0.2500, sp: 0.8095, f1: 0.1118
                                   tn: 493, tp: 9, fn: 27, fp: 116
Epoch: 19
  Train loss: 0.8157, acc: 0.5227, pr: 0.3635, rc: 0.7989, sp: 0.4054, f1: 0.4996
                                   tn: 3714, tp: 3111, fn: 783, fp: 5448
  Test  loss: 1.0305, acc: 0.3178, pr: 0.0491, rc: 0.6111, sp: 0.3005, f1: 0.0909
                                   tn: 183, tp: 22, fn: 14, fp: 426
Epoch: 20
  Train loss: 0.6819, acc: 0.6211, pr: 0.4160, rc: 0.6692, sp: 0.6006, f1: 0.5130
                                   tn: 5503, tp: 2606, fn: 1288, fp: 3659
  Test  loss: 0.7731, acc: 0.5271, pr: 0.0561, rc: 0.4722, sp: 0.5304, f1: 0.1003
                                   tn: 323, tp: 17, fn: 19, fp: 286
Epoch: 21
  Train loss: 0.6411, acc: 0.6730, pr: 0.4619, rc: 0.5

  Train loss: 0.5216, acc: 0.7739, pr: 0.6185, rc: 0.6312, sp: 0.8345, f1: 0.6248
                                   tn: 7646, tp: 2458, fn: 1436, fp: 1516
  Test  loss: 0.4899, acc: 0.8388, pr: 0.1458, rc: 0.3889, sp: 0.8654, f1: 0.2121
                                   tn: 527, tp: 14, fn: 22, fp: 82
Epoch: 46
  Train loss: 0.4983, acc: 0.7885, pr: 0.6725, rc: 0.5673, sp: 0.8826, f1: 0.6154
                                   tn: 8086, tp: 2209, fn: 1685, fp: 1076
  Test  loss: 0.4343, acc: 0.8744, pr: 0.1918, rc: 0.3889, sp: 0.9031, f1: 0.2569
                                   tn: 550, tp: 14, fn: 22, fp: 59
Epoch: 47
  Train loss: 0.4930, acc: 0.7893, pr: 0.6849, rc: 0.5437, sp: 0.8937, f1: 0.6062
                                   tn: 8188, tp: 2117, fn: 1777, fp: 974
  Test  loss: 0.4313, acc: 0.8760, pr: 0.1765, rc: 0.3333, sp: 0.9080, f1: 0.2308
                                   tn: 553, tp: 12, fn: 24, fp: 56
Epoch: 48
  Train loss: 0.4732, acc: 0.8012, pr: 0.7133, rc: 0.557

In [9]:
test_result = np.array(test_result)
print(test_result)

[[0.00000000e+00 1.42637718e+00 3.10077518e-02 0.00000000e+00
  3.60000000e+01 0.00000000e+00 6.09000000e+02 5.58139535e-02
  1.00000000e+00 0.00000000e+00 1.05726872e-01]
 [1.00000000e+00 1.43103886e+00 3.72093022e-02 1.00000000e+00
  3.60000000e+01 0.00000000e+00 6.08000000e+02 5.59006211e-02
  1.00000000e+00 1.64203612e-03 1.05882353e-01]
 [2.00000000e+00 1.52142847e+00 3.10077518e-02 0.00000000e+00
  3.60000000e+01 0.00000000e+00 6.09000000e+02 5.58139535e-02
  1.00000000e+00 0.00000000e+00 1.05726872e-01]
 [3.00000000e+00 1.70276618e+00 3.72093022e-02 3.00000000e+00
  3.60000000e+01 0.00000000e+00 6.06000000e+02 5.60747664e-02
  1.00000000e+00 4.92610837e-03 1.06194690e-01]
 [4.00000000e+00 1.42516303e+00 3.72093022e-02 0.00000000e+00
  3.60000000e+01 0.00000000e+00 6.09000000e+02 5.58139535e-02
  1.00000000e+00 0.00000000e+00 1.05726872e-01]
 [5.00000000e+00 1.54905903e+00 3.41085270e-02 0.00000000e+00
  3.60000000e+01 0.00000000e+00 6.09000000e+02 5.58139535e-02
  1.00000000e+00